# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import tensorflow as tf


from matplotlib import pyplot as plt
import glob as glob
import random
import pickle

import sys
import cv2
import csv

%matplotlib inline
# tf.executing_eagerly()

In [ ]:
score_threshold = 0.3

# Environment setup

In [ ]:


sys.path.append('/Users/yuxing/traffic-sign-detection/models/research')  # Replace with the path to TensorFlow Object Detection API
# sys.path.append('../darkflow')  # Replace with the path to Darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


## Model preparation 

# Tensorflow Object Detection API

In [ ]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb3_label_map.pbtxt')

NUM_CLASSES = 3


## Load a (frozen) Tensorflow model into memory

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [ ]:
PATH_TO_LABELS = '/Users/yuxing/traffic-sign-detection/gtsdb_data/gtsdb_label_map.pbtxt'

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
# print(label_map)


## Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    #return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    return np.array(image.getdata()).reshape((im_height, im_width, 3))

## Detection

In [ ]:

PATH_TO_TEST_IMAGES_DIR = '/Users/yuxing/traffic-sign-detection/test_images'

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

In [ ]:

def append_results(results, image_name, iteration, class_name, score, box, sum_diff, max_diff):
    results.append({
        'image_name': image_name,
        'iteration': iteration + 1,
        'class': class_name,
        'score': float(score),
        'box': box,
        'total_perturbation': sum_diff,
        'max_perturbation': max_diff
    })

def save_results_to_csv(results, image_index, save_dir, algorithm_number):
    csv_filename = f"adv_alg{algorithm_number}_results.csv"
    csv_path = os.path.join(save_dir, csv_filename)
    
    # Check if file exists
    file_exists = os.path.isfile(csv_path)
    
    with open(csv_path, 'a', newline='') as csvfile:
        # Include 'image_index' in the fieldnames
        fieldnames = ['image_index', 'image_name', 'iteration', 'class', 'score', 'box', 'total_perturbation', 'max_perturbation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
   
        if not file_exists:
            writer.writeheader()
        
        for result in results:
            # Include image_index
            result['image_index'] = image_index + 1
            writer.writerow(result)

# # Run inference and visualization
# with detection_graph.as_default():
#     with tf.Session(graph=detection_graph) as sess:
#         for idx, image_path in enumerate(TEST_IMAGE_PATHS):
#             image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
#             boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
#             scores = detection_graph.get_tensor_by_name('detection_scores:0')
#             classes = detection_graph.get_tensor_by_name('detection_classes:0')
#             num_detections = detection_graph.get_tensor_by_name('num_detections:0')

#             image_data = tf.read_file(image_path)
#             image_decoded = tf.image.decode_image(image_data)
#             image_np = sess.run(image_decoded)
#             image_np_expanded = np.expand_dims(image_np, axis=0)

#             (boxes, scores, classes, num_detections) = sess.run(
#                 [boxes, scores, classes, num_detections],
#                 feed_dict={image_tensor: image_np_expanded})
            
#             vis_util.visualize_boxes_and_labels_on_image_array(
#                 image_np,
#                 np.squeeze(boxes),
#                 np.squeeze(classes).astype(np.int32),
#                 np.squeeze(scores),
#                 category_index,
#                 use_normalized_coordinates=True,
#                 line_thickness=6)
#             plt.figure(idx, figsize=IMAGE_SIZE)
#             plt.axis('off')
#             plt.imshow(image_np)



In [ ]:
algorithm_number = 0  # Adjust this for different algorithms

SAVE_DIR = 'test_image_results'
os.makedirs(SAVE_DIR, exist_ok=True)

# Run inference and visualization
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            image_np_expanded = np.expand_dims(image_np, axis=0)

            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            # Get the image name from the image path
            image_name = os.path.basename(image_path)

            # Record the original image detections
            original_results = []
            for i in range(len(np.squeeze(scores))):
                if np.squeeze(scores)[i] > score_threshold:
                    class_id = int(np.squeeze(classes)[i])
                    class_name = category_index[class_id]['name']
                    box = np.squeeze(boxes)[i].tolist()
                    append_results(original_results, image_name, -1, class_name, np.squeeze(scores)[i], box, 0, 0)
            
            # Save original image results to CSV
            save_results_to_csv(original_results, idx, SAVE_DIR, algorithm_number)

            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            plt.figure(idx, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(image_np)


# Functions

In [ ]:
def region_downsample(image_region, factor):
    return cv2.resize(image_region, None, fx=factor, fy=factor, interpolation=cv2.INTER_LINEAR)

def region_upsample(image_region, target_shape):
    return cv2.resize(image_region, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR)

def compute_loss(sess, image_np_expanded, detection_graph):
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    
    (scores, classes) = sess.run([scores, classes], feed_dict={image_tensor: image_np_expanded})
    max_score_index = np.argmax(scores)
    score = scores[0][max_score_index]
    approx_logits = np.log(score / (1 - score))
    return approx_logits

def approximate_gradient(sess, image_np_expanded, detection_graph, h, num_samples):
    original_loss = compute_loss(sess, image_np_expanded, detection_graph)
    grad_approx = np.zeros_like(image_np_expanded).astype(np.float32)
    
    total_pixels = np.prod(image_np_expanded.shape)
    num_samples = min(num_samples, total_pixels)  # Adjust the num_samples value if total_pixels is less than NUM_SAMPLES
    indices = np.random.choice(total_pixels, num_samples, replace=False)  # Randomly selecting pixel indices from the entire image
        
    for flat_idx in indices:
        idx = np.unravel_index(flat_idx, image_np_expanded.shape)
        perturb = np.zeros_like(image_np_expanded)
        perturb[idx] = h
        perturbed_image = image_np_expanded + perturb
        perturbed_loss = compute_loss(sess, perturbed_image, detection_graph)
        grad_approx[idx] = (perturbed_loss - original_loss) / h 

    return grad_approx

def save_adversarial_image(image, idx, iteration, save_dir, algorithm_number):
    image_name = f"alg{algorithm_number}_{idx + 1:03d}_itr{iteration + 1}.jpg"
    image_save_path = os.path.join(save_dir, image_name)
    plt.imsave(image_save_path, image)
    return image_name

def save_adversarial_image(image, idx, iteration, save_dir, algorithm_number):
    image_name = f"alg{algorithm_number}_{idx + 1:03d}_itr{iteration + 1}.jpg"
    image_save_path = os.path.join(save_dir, image_name)
    plt.imsave(image_save_path, image)
    return image_name

def process_bounding_boxes(sess, adv_image, boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor, image_tensor, score_threshold, buffer_factor, original_shape, factor, num_samples, h):
    grad_approx = np.zeros_like(adv_image).astype(np.float32)
    
    # Fetch the original detections to compute bounding boxes
    (initial_boxes, initial_scores, _, _) = sess.run(
        [boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor],
        feed_dict={image_tensor: np.expand_dims(adv_image, axis=0)}
    )
    initial_boxes = np.squeeze(initial_boxes)
    initial_scores = np.squeeze(initial_scores)

    valid_indices = np.where(initial_scores > score_threshold)[0]  # Filter detections based on threshold

    # For each valid detection
    for valid_idx in valid_indices:
        valid_box = initial_boxes[valid_idx]
        ymin, xmin, ymax, xmax = valid_box
        height, width = ymax - ymin, xmax - xmin

        ymin = max(0, ymin - buffer_factor * height)
        xmin = max(0, xmin - buffer_factor * width)
        ymax = min(1, ymax + buffer_factor * height)
        xmax = min(1, xmax + buffer_factor * width)

        ymin, xmin, ymax, xmax = int(ymin * original_shape[0]), int(xmin * original_shape[1]), int(ymax * original_shape[0]), int(xmax * original_shape[1])

        if ymax - ymin > 0 and xmax - xmin > 0:
            region = adv_image[ymin:ymax, xmin:xmax]
            region_low_res = region_downsample(region, factor)
            region_low_res_expanded = np.expand_dims(region_low_res, axis=0)

            # Compute the approximate gradient on low-res region
            region_grad_approx_low_res = approximate_gradient(sess, region_low_res_expanded, detection_graph, h, num_samples)
            # Upsample gradient back to original resolution of that region
            region_grad_approx = region_upsample(region_grad_approx_low_res[0], region.shape)

            grad_approx[ymin:ymax, xmin:xmax] = region_grad_approx

    return grad_approx


# Gradient Based Method (adversarial pattern only generated in and around bounding boxes)

In [ ]:
# Tuning parameters
FACTOR = 0.4
H = 10
NUM_SAMPLES = 50
NUM_ITERATIONS = 10
EPS = 40000
BUFFER_FACTOR =1
algorithm_number = 1

SAVE_DIR = 'saved_images_1'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                perturbed_adv_image = adv_image + EPS * grad_approx  # Perturb the image

                # Calculate the total and max perturbation before clipping
                per_box_diff = perturbed_adv_image - last_adv_image
                sum_diff = np.sum(np.abs(per_box_diff), axis=(0, 1, 2))
                max_diff = np.max(np.abs(per_box_diff), axis=(0, 1, 2))

                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Update the last_adv_image with the perturbed values before clipping
                last_adv_image = perturbed_adv_image.copy()

                # Apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)


                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)

# Hard Clipping Gradient Based Method (adversarial pattern only generated in and around bounding boxes)

In [ ]:
# Tuning parameters
FACTOR = 0.4
H = 10
NUM_SAMPLES = 50
NUM_ITERATIONS = 10
EPS = 40000
BUFFER_FACTOR =1
algorithm_number = 3
SAT = 10000

SAVE_DIR = 'saved_images_3'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                perturbed_adv_image = adv_image + np.clip(EPS * grad_approx, -SAT, SAT)

                # Calculate the total and max perturbation before clipping
                per_box_diff = perturbed_adv_image - last_adv_image
                sum_diff = np.sum(np.abs(per_box_diff), axis=(0, 1, 2))
                max_diff = np.max(np.abs(per_box_diff), axis=(0, 1, 2))

                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Update the last_adv_image with the perturbed values before clipping
                last_adv_image = perturbed_adv_image.copy()

                # Apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)


                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)

# Projected Gradient Descent Method (adversarial pattern only generated in and around bounding boxes)

In [ ]:
def project_perturbation(perturbation, epsilon, p_norm='inf'):
    """
    Project the perturbation onto an epsilon ball.
    """
    if p_norm == 'inf':
        return np.clip(perturbation, -epsilon, epsilon)
    elif p_norm == 2:
        norm = np.linalg.norm(perturbation)
        if norm > epsilon:
            return perturbation * (epsilon / norm)
        return perturbation
    else:
        raise NotImplementedError("Only L-inf and L-2 norms are supported")
    



## 1. l_inf Norm

In [ ]:
# Tuning parameters
FACTOR = 0.4
H = 10
NUM_SAMPLES = 50
NUM_ITERATIONS = 10
EPS = 40000
BUFFER_FACTOR =1
algorithm_number = 5
epsilon = 10000

# Define the directory for saving images and results
SAVE_DIR = 'saved_images_5'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                # Project the perturbation onto the epsilon ball
                perturbation = EPS * grad_approx
                perturbation = project_perturbation(perturbation, epsilon)
                # perturbation = project_perturbation(perturbation, epsilon, p_norm= 2)

                # adv_image = adv_image.astype(np.float32)
                perturbed_adv_image = adv_image + EPS * grad_approx 

                # Calculate the total and max perturbation before clipping
                per_box_diff = perturbed_adv_image - last_adv_image
                sum_diff = np.sum(np.abs(per_box_diff), axis=(0, 1, 2))
                max_diff = np.max(np.abs(per_box_diff), axis=(0, 1, 2))

                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Now apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Update the last_adv_image with the perturbed values before clipping
                last_adv_image = perturbed_adv_image.copy()

                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)

                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)


## 2. l_2 Norm

In [ ]:
# Tuning parameters
FACTOR = 0.4
H = 10
NUM_SAMPLES = 50
NUM_ITERATIONS = 10
EPS = 40000
BUFFER_FACTOR =1
algorithm_number = 6
epsilon = 10000

# Define the directory for saving images and results
SAVE_DIR = 'saved_images_6'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                # Project the perturbation onto the epsilon ball
                perturbation = EPS * grad_approx
                perturbation = project_perturbation(perturbation, epsilon, p_norm= 2)

                # adv_image = adv_image.astype(np.float32)
                perturbed_adv_image = adv_image + EPS * grad_approx 

                # Calculate the total and max perturbation before clipping
                per_box_diff = perturbed_adv_image - last_adv_image
                sum_diff = np.sum(np.abs(per_box_diff), axis=(0, 1, 2))
                max_diff = np.max(np.abs(per_box_diff), axis=(0, 1, 2))

                # Print out the desired information after the perturbation and before clipping
                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Now apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Update the last_adv_image with the perturbed values before clipping
                last_adv_image = perturbed_adv_image.copy()

                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)

                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)
